# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING DATA FILES

#### Import Python packages 

In [13]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [14]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [16]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion. 

## The <font color=red>event_datafile_new.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [17]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [18]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [19]:
session.set_keyspace("udacity")


## Create queries to ask the following three questions of the data

### Query 1: Select artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4  
The table **music_library_by_sessions**, the **sessionid** column was used as a PARTITION KEY to filter the result by this column. We have used **iteminsession** column as clustering columns to sort the result.

In [20]:
session.execute("""
CREATE TABLE IF NOT EXISTS music_library_by_sessions (sessionid INT, iteminsession INT, artist TEXT, \
song TEXT, length FLOAT,
PRIMARY KEY (sessionid, iteminsession))
""")                  

In [21]:
#Read .csv file and extract required rows to insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_sessions (sessionid, iteminsession, artist, song, length) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [22]:
query = "SELECT artist, song, length FROM music_library_by_sessions WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Select name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
The table **music_library_by_users**, the **userid** and **sessionid** columns were used as a PARTITION KEY to filter the result by this column. We have used **iteminsession** column as clustering columns to sort the result.

In [23]:
session.execute("""
CREATE TABLE IF NOT EXISTS music_library_by_users (userid INT, sessionid INT, iteminsession INT, \
firstname TEXT, lastname TEXT, artist TEXT, song TEXT, 
PRIMARY KEY ((userid, sessionid), iteminsession))
""")

In [24]:
#Read .csv file and extract required rows to insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_users (userid, sessionid, iteminsession, \
        firstname, lastname, artist, song) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [25]:
query = "SELECT artist, song, firstname, lastname FROM music_library_by_users WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Select user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
The table **music_library_by_song_name**, the **song** column was used as a PARTITION KEY to filter the result by this column. We have used **userid** column as clustering column to sort the result.

In [26]:
session.execute("""
CREATE TABLE IF NOT EXISTS music_library_by_song_name (song TEXT, user_id INT, firstname TEXT, lastname TEXT,
PRIMARY KEY ((song), user_id))
""")

In [27]:
#Read .csv file and extract required rows to insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_song_name (song, user_id, firstname, lastname)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [28]:
query = "SELECT firstname, lastname, song FROM music_library_by_song_name WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
for row in rows:
    print (row.firstname, row.lastname, row.song)

Jacqueline Lynch All Hands Against His Own
Tegan Levine All Hands Against His Own
Sara Johnson All Hands Against His Own


### Drop the tables before closing out the sessions

In [29]:
## Drop the table before closing out the sessions

In [30]:
session.execute("DROP TABLE music_library_by_sessions")

In [31]:
session.execute("DROP TABLE music_library_by_users")

In [32]:
session.execute("DROP TABLE music_library_by_song_name")

### Close the session and cluster connectionÂ¶

In [33]:
session.shutdown()
cluster.shutdown()